In [1]:
import math
import numpy as np
from PyCO2SYS import CO2SYS
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# calccomp    This function calculates the ionic strength and composition of background electrolyte 
#             i.e standard seawater [Millero et al., 2008]. On a molar concentration 
#             basis, seven chemical elements account for 99.9% of the dissolved species in seawater. 
#             SHIAMI defines the background seawater electrolyte containing 12 elements 
#             whose concentrations exceed 1?molkg?1: 
#             
#             % Na+, K+, Mg2+, Ca2+, Sr2+, Cl-, SO42-, CO32- HCO3-  Br-, F-, B(OH)4 
##            Ionic strength 
#             Like Hain et al, SHIAMI uses modern seawater relationships among 
#             Practical Salinity Scale salinity (UNESCO, 1981), ionic strength (I), and chlorinity(Cl) 
#             (Dickson, 2010) ---> {Denoted by *}
#
#              I  = 19.924.*S./(1000-1.005.*S); %  Ionic strength*: Determined from Salinity
##            Determining concentration:   
#             Concentration given in mol of solute/kg of seawater
##            For Major ions except carbonate and bicarbonate ions: 
#             FROM: Millero et al., 2008
#             Determined from salinity 
# 
#             In the open ocean, the principal chemical constituents in 
#             seawater have nearly identical concentrations relative to one another.  
#             Consequently, by determining the concentration of one major chemical 
#             constituent of seawater, you can determine the concentrations of each 
#             individual principal chemical element present in seawater.
#             The constituent originally selected to characterize a given sample of 
#             seawater or other natural waters is chlorinity, Cl (). 
#
#             SHIAMI uses the following relationship between chlorinity* and salinity:
#                  S() = 1.80655 x Chlorinity () 
#                  Cl(mol/kg) = 0.5458696 * S/35
#             ************************************************************************
#             FOR EXAMPLE:
#             [Na] = 0.5564924 {g of solute /Kg of solvent} / Cl() .... (1)                
#             0.5564924 * 1/(22.98976928)  mol/g  = 0.024211 .... (2)         
#             0.024211 * Cl  = 0.024211 * S/1.80655 = (0.0134*S) mol/kg  
#             Using S = 35.000: 
#             [Na] =  0.46897 mol/kg of solvent ... (3)                                               
#             0.46897 * (S/1.80655)/(35/1.80655)  = 0.46897 * (S/35)
#
#             (1) Table 4, column 5; Millero, 2008
#             (2) Inverse of Table 4, column 6; Millero, 2008
#             (3) Table 4, column 7; Millero, 2008
#            **************************************************************************
##            For Carbonate system
#             SHIAMI uses the CO2SYS model to output the total concentration 
#             of HCO3, CO3 and OH (umol/kgSW). 
# 
#             MIAMI and MyAMI relates total concentration of HCO3- and CO32- to salinity.  The use of 
#             salinity limits the models use for waters of different compositions (Turner, et al 2016). 
#             
#             SHIAMI use of the MATLAB version of CO2SYS (Lewis et al. 1998; Van Heuven et al. 2011). 
#             Provided with sufficient input, CO2SYS returns the state of the carbonate system  of the 
#             water sample. Specifically, SHIAMI uses the CO2SYS model to output the total concentration 
#             of HCO3, CO3 and OH (umol/kgSW). 
# 
#             To characterize the various components of the carbonate system using CO2SYS one must measure, 
#             at least two of the following:
#
#             1)	pH
#             2)	Alkalinity, (TA)
#             3)	Total CO2, (TCO2)
#             4)	Partial pressure of CO2, (pCO2)
#
#             And both: 
#
#             1) Concentration of silicate  
#             2) Concentration of phosphate 
#          
# 
#             INPUT:
#             PAR1  (some unit) : scalar or vector of size n x 1
#             PAR2  (some unit) : scalar or vector of size n x 1
#             PAR1TYPE       () : scalar or vector of size n x 1(*)
#             PAR2TYPE       () : scalar or vector of size n x 1(*)
#             S              () : scalar or vector of size n x 1
#             Tc      (degr. C) : scalar or vector of size n x 1
#             P         ( dbar) : scalar or vector of size n x 1
#             SIL    (umol/kgSW): scalar or vector of size n x 1
#             PO4    (umol/kgSW): scalar or vector of size n x 1
#             pHSCALE           : scalar or vector of size n x 1(**)
#
#            (*) Each element must be an integer, 
#             indicating that PAR1 (or PAR2) is of type: 
#             1 = Total Alkalinity
#             2 = DIC
#             3 = pH
#             4 = pCO2
#             5 = fCO2
# 
#            (**) Each element must be an integer, 
#             indicating that the pH-input (PAR1 or PAR2, if any) is at:
#             1 = Total scale
#             2 = Seawater scale
#             3 = Free scale
#             4 = NBS scale
#
#            Uses: K1 K2 dissociation constants -> Millero et al, 2010
#                  KSO4 dissociation constants  -> Dickson & TB of Uppstrom 1979  (PREFERRED) 
#                  
#            If one wants to use other dissociation constants, 
#            edit values for k1k2c, kso4c in line in the function file
#            using the following (from CO2SYS.m) :       
#        
#            K1 K2 dissociation constants that are to be used:
#            1 = Roy, 1993											    T:    0-45  S:  5-45. Total scale. Artificial seawater.
#            2 = Goyet & Poisson										T:   -1-40  S: 10-50. Seaw. scale. Artificial seawater.
#            3 = HANSSON              refit BY DICKSON AND MILLERO	    T:    2-35  S: 20-40. Seaw. scale. Artificial seawater.
#            4 = MEHRBACH             refit BY DICKSON AND MILLERO	    T:    2-35  S: 20-40. Seaw. scale. Artificial seawater.
#            5 = HANSSON and MEHRBACH refit BY DICKSON AND MILLERO	    T:    2-35  S: 20-40. Seaw. scale. Artificial seawater.
#            6 = GEOSECS (i.e., original Mehrbach)					    T:    2-35  S: 19-43. NBS scale.   Real seawater.
#            7 = Peng	(i.e., originam Mehrbach but without XXX)	    T:    2-35  S: 19-43. NBS scale.   Real seawater.
#            8 = Millero, 1979, FOR PURE WATER ONLY (i.e., Sal=0)	    T:    0-50  S:     0. 
#            9 = Cai and Wang, 1998									    T:    2-35  S:  0-49. NBS scale.   Real and artificial seawater.
#           10 = Lueker et al, 2000									    T:    2-35  S: 19-43. Total scale. Real seawater.
#           11 = Mojica Prieto and Millero, 2002.					    T:    0-45  S:  5-42. Seaw. scale. Real seawater
#           12 = Millero et al, 2002									T: -1.6-35  S: 34-37. Seaw. scale. Field measurements.
#           13 = Millero et al, 2006									T:    0-50  S:  1-50. Seaw. scale. Real seawater.
#           14 = Millero et al, 2010									T:    0-50  S:  1-50. Seaw. scale. Real seawater.
# 
#           KSO4 dissociation constants in combination with the formulation
#           of the borate-to-salinity ratio:
#           (Having both these choices in a single argument is somewhat awkward, 
#           but it maintains syntax compatibility with the previous
#           version.)
#           1 = KSO4 of Dickson & TB of Uppstrom 1979  (PREFERRED) 
#           2 = KSO4 of Khoo    & TB of Uppstrom 1979
#           3 = KSO4 of Dickson & TB of Lee 2010
#           4 = KSO4 of Khoo    & TB of Lee 2010
#          FOR MORE INFO: SEE CO2SYS.m 

## --------------------------------------------------------------------------------------
##  Assign values to global variables
mjCaMax = 5         #In python array goes 0,1,2,3,4,5,6, so mjCaMax must go to 7 to include the 6th variable
mnCaMax = 37
CaMax = mjCaMax + mnCaMax

mjAnMax = 7         #In python array goes 0,1,2,3,4,5,6,7,8 so mjCaMax must go to 8 to include the 7th variable
mnAnMax = 19
AnMax = mjAnMax + mnAnMax

NeutMax = 7

# S  = 35                         #        Salinity*: 0-40 psu, Practical Salinity Scale salinity (UNESCO, 1981)
# I  = 19.924*S/(1000-1.005*S)    #        Ionic strength*: Determined from Salinity
# Tc = 25                         #        Temperature: 0-50 degrees C
# P  = 1                          #        Pressure: Via CO2SYS, Information on pressure dependence is limited
# pH = 8.1                        #        pH: seawater scale 

# par1type = 1                    # The first parameter supplied is of type "1", which is "alkalinity"
# par1     = 2400                 # value of the first parameter
# par2type = 3                    # The first parameter supplied is of type "3", which is "pH"
# par2     = pH                   # value of the second parameter
# sal      = S                    # Salinity of the sample
# tempin   = Tc                   # Temperature at input conditions
# presin   = 10.1325              # Pressure    at input conditions
# tempout  = np.nan               # Temperature at output conditions - doesn't matter here
# presout  = np.nan               # Pressure    at output conditions - doesn't matter here
# sil      = 50                   # Concentration of silicate  in the sample (in umol/kg)
# po4      = 2                    # Concentration of phosphate in the sample (in umol/kg)
# pHscale  = 2                    # pH scale at which the input pH is reported ("1" means "Seawater Scale")
# k1k2c    = 14                   # Choice of H2CO3 and HCO3- dissociation constants K1 and K2 ("14" means "Millero 2010")
# kso4c    = 1                    # Choice of HSO4- dissociation constants KSO4 ("1" means "Dickson")

## --------------------------------------------------------------------------------------
def calccomp (S, Tc, P, PAR1, PAR2, PAR1TYPE, PAR2TYPE, SIL, PO4, pHSCALE):
    
    ## Ionic strength
    I  = 19.924*S/(1000-1.005*S)
    
    ## Cations 
    mT_cat[0] = 0.4689674*S/35         # [Na]           
    mT_cat[1] = 0.0102077*S/35         # [K]
    mT_cat[2] = 0.0528171*S/35         # [Mg]
    mT_cat[3] = 0.0102821*S/35         # [Ca]      
    mT_cat[4] = 0.0000907*S/35         # [Sr]
    
    ## FOR MORE INFO: SEE CO2SYS.m  
    TEMPOUT  = np.nan      # Temperature at output conditions - doesn't matter here
    PRESOUT  = np.nan      # Pressure    at output conditions - doesn't matter here
    k1k2c    = 14          # Choice of H2CO3 and HCO3- dissociation constants K1 and K2 ("14" means "Millero 2010")
    kso4c    = 1           # Choice of HSO4- dissociation constants KSO4 ("1" means "Dickson")


    A = CO2SYS(PAR1, PAR2, PAR1TYPE, PAR2TYPE, S, Tc, TEMPOUT, P, PRESOUT, SIL, PO4, pHSCALE, k1k2c, kso4c)
    # FOR MORE INFO: SEE CO2SYS.m  
    
    ## --------------------------------------------------------------------------------------------------------------

    ##  2c) Anions
    mT_an[0]  = 0.5458696*S/35          # [Cl]
    mT_an[1]  = 0.0282352*S/35          # [SO4]
    mT_an[2]  = 0.00024694*S/35         # [CO3]  : A[6] * 1e-6   :   0.0002390*S/35  :    0.000246937727254629       
    mT_an[3]  = 0.00179143*S/35         # [HCO3] : A[5] * 1e-6   :   0.0017177*S/35  :    0.001791429070585
    mT_an[4]  = 0.0008421*S/35          # [Br]
    mT_an[5]  = 0.0000683*S/35          # [F]
    mT_an[6]  = 0.0001008*S/35          # [B(OH)4] 
    OH = 10**(-14+8.1)                  # A[9] * 1e-6 
    
    return I, mT_cat, mT_an, OH